# Tugas Kecil 1
## Eksplorasi library Decision Tree Learning pada Jupyter Notebook

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Modul-modul pengolahan data
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Modul model algoritma machine learning
from sklearn.tree import DecisionTreeClassifier, export_text, export_graphviz

# Modul pengecekan akurasi
from sklearn.metrics import accuracy_score, f1_score

In [2]:
breast_cancer = load_breast_cancer()
df_breast_cancer = pd.DataFrame(breast_cancer.data, 
                                columns=breast_cancer.feature_names)
df_breast_cancer['target'] = breast_cancer.target
df_breast_cancer.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [3]:
df_play_tennis = pd.read_csv('PlayTennis.csv')

le = LabelEncoder()
df_play_tennis['Play Tennis'] = le.fit_transform(df_play_tennis['Play Tennis'])

df_play_tennis.head()

,Outlook,Temperature,Humidity,Wind,Play Tennis
0,Sunny,Hot,High,Weak,0
1,Sunny,Hot,High,Strong,0
2,Overcast,Hot,High,Weak,1
3,Rain,Mild,High,Weak,1
4,Rain,Cool,Normal,Weak,1


In [4]:
X_breast_cancer = df_breast_cancer[breast_cancer.feature_names]
y_breast_cancer = df_breast_cancer['target']

In [5]:
X_bc_train, X_bc_test, y_bc_train, y_bc_test = train_test_split(
    X_breast_cancer, y_breast_cancer, test_size=0.2, random_state=42)

In [6]:
X_play_tennis = df_play_tennis[['Outlook', 'Temperature', 'Humidity', 'Wind']]
y_play_tennis = df_play_tennis['Play Tennis']

In [7]:
X_pt_train, X_pt_test, y_pt_train, y_pt_test = train_test_split(
    X_play_tennis, y_play_tennis, test_size=0.2, random_state=42)

### Decision Tree Classifier

#### 1. Dataset Breast Cancer

In [21]:
decision_tree = DecisionTreeClassifier(random_state=42)
decision_tree = decision_tree.fit(X_bc_train, y_bc_train)

r = export_text(decision_tree, feature_names=breast_cancer['feature_names'].tolist())
print(r)

|--- mean concave points <= 0.05
|   |--- worst radius <= 16.83
|   |   |--- area error <= 48.70
|   |   |   |--- worst smoothness <= 0.18
|   |   |   |   |--- smoothness error <= 0.00
|   |   |   |   |   |--- worst texture <= 27.76
|   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |--- worst texture >  27.76
|   |   |   |   |   |   |--- class: 0
|   |   |   |   |--- smoothness error >  0.00
|   |   |   |   |   |--- worst texture <= 33.35
|   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |--- worst texture >  33.35
|   |   |   |   |   |   |--- worst texture <= 33.56
|   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |--- worst texture >  33.56
|   |   |   |   |   |   |   |--- class: 1
|   |   |   |--- worst smoothness >  0.18
|   |   |   |   |--- class: 0
|   |   |--- area error >  48.70
|   |   |   |--- concavity error <= 0.02
|   |   |   |   |--- class: 0
|   |   |   |--- concavity error >  0.02
|   |   |   |   |--- class: 1
|   |--- worst radius >  16.

In [38]:
#visualisasi tree
#import graphviz

#dot_data = export_graphviz(decision_tree, out_file=None) 
#graph = graphviz.Source(dot_data) 


In [37]:
# Memprediksi hasil dari model
y_bc_pred = decision_tree.predict(X_bc_test)

# Mengevaluasi hasil prediksi
# Menggunakan metric Accuracy
print("Evaluasi hasil prediksi menggunakan metric Accuracy: {}".format(accuracy_score(y_bc_test, y_bc_pred)))

# Menggunakan metric F1
print("Evaluasi hasil prediksi menggunakan metric F1: {}".format(f1_score(y_bc_test, y_bc_pred)))

Evaluasi hasil prediksi menggunakan metric Accuracy: 0.9473684210526315
Evaluasi hasil prediksi menggunakan metric F1: 0.9577464788732394


#### 2. Dataset Play Tennis

### Id3Estimator

In [ ]:
pass

#### 1. Dataset Breast Cancer

#### 2. Dataset Play Tennis

### K Means

In [ ]:
pass

#### 1. Dataset Breast Cancer

#### 2. Dataset Play Tennis

### LogisticRegression

In [ ]:
pass

#### 1. Dataset Breast Cancer

#### 2. Dataset Play Tennis

### Neural Network

In [ ]:
pass

#### 1. Dataset Breast Cancer

#### 2. Dataset Play Tennis

### SVM

In [ ]:
pass

#### 1. Dataset Breast Cancer

#### 2. Dataset Play Tennis

Pembagian Tugas
- Raihan Astrada (1,2,3 dataset breast cancer & 4,5,6 dataset play tennis)
- Daffa Ananda (4,5,6 dataset breast cancer & 1,2,3 dataset play tennis)